### Step1 Get transcripts for lncRNA genes.

In [29]:
# Human
import pandas as pd
import os
import re

# Set the directory containing ensembl transcript data
ensembl_dir = "../../reference_lncRNA/human/transcript/ensembl/"

# Read LncBook and NONCODE transcript files
cols = ['gene_id', 'transcript_id']
noncodev5_trans = pd.read_csv('../../reference_lncRNA/human/transcript/NONCODEv5_human_hg38_lncRNA_trans.txt', sep='\t', header=None, names=cols)
noncodev6_trans = pd.read_csv('../../reference_lncRNA/human/transcript/NONCODEv6_human_hg38_lncRNA_trans.txt', sep='\t', header=None, names=cols)

# Read lncRNA ID data
lncRNA = pd.read_csv('../../data/LPI/human/lncRNA.csv')
lncRNA = lncRNA[['lncRNA_id','gene_id','gene_name', 'identifier']]

# Initialize remaining lncRNA list
remained_lncRNA = lncRNA[['lncRNA_id','gene_id','gene_name', 'identifier']].copy()

# Function to remove matched rows
def update_remained_lncRNA(df, matched_df):
    """Remove matched lncRNA_id from the remaining lncRNA list"""
    updated_df = df[~df['lncRNA_id'].isin(matched_df['lncRNA_id'])]
    return updated_df[['lncRNA_id','gene_id','gene_name', 'identifier']]

# Store results
results = []

# Get transcript by NONCODE (v6 and v5)
trans_lnc_noncodev6 = pd.merge(remained_lncRNA, noncodev6_trans, left_on='identifier', right_on='gene_id', how='inner')
results.append(trans_lnc_noncodev6)
remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_noncodev6)

trans_lnc_noncodev5 = pd.merge(remained_lncRNA, noncodev5_trans, left_on='identifier', right_on='gene_id', how='inner')
results.append(trans_lnc_noncodev5)
remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_noncodev5)

# Extract version numbers and sort filenames in descending order
def extract_version(filename):
    match = re.search(r'GRCh38\.(\d+)_trans\.txt', filename)
    #match = re.search(r"v(\d+)", filename) # ensembl
    return int(match.group(1)) if match else -1  # Extract version number, default to -1 if no match

ensembl_files = [f for f in os.listdir(ensembl_dir) if f.endswith(".txt")]
sorted_ensembl_files = sorted(ensembl_files, key=extract_version, reverse=True)  # Sort by version number (descending)

# Iterate through sorted ensembl transcript files
for txt_file in sorted_ensembl_files:
    file_path = os.path.join(ensembl_dir, txt_file)

    # Read ensembl transcript file
    ensembl_trans = pd.read_csv(
        file_path, sep='\t', header=None,
        names=['gene_id', 'gene_name', 'transcript_id']
    )

    trans_lnc_ensembl = pd.merge(
        remained_lncRNA,
        ensembl_trans[['gene_id', 'transcript_id']],
        left_on='identifier', right_on='gene_id', how='inner'
    )
    results.append(trans_lnc_ensembl)
    remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_ensembl)
    
for txt_file in sorted_ensembl_files:
    file_path = os.path.join(ensembl_dir, txt_file)

    # Read ensembl transcript file
    ensembl_trans = pd.read_csv(
        file_path, sep='\t', header=None,
        names=['gene_id', 'gene_name', 'transcript_id']
    )
    
    trans_lnc_symbol = pd.merge(
        remained_lncRNA,
        ensembl_trans[['gene_name', 'transcript_id']],
        left_on='identifier', right_on='gene_name', how='inner'
    )
    results.append(trans_lnc_symbol)

    # --- Update remained set ---
    remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_symbol)

remained_lncRNA.to_csv("no_trans.csv", index=False)

# Combine all results into a single DataFrame
trans_lnc = pd.concat(results, ignore_index=True)
trans_lnc = trans_lnc[['lncRNA_id', 'transcript_id']]

# Save results to file
trans_lnc.drop_duplicates().to_csv('./human/lnc_trans.csv', index=False, header=['lncRNA_id','transcript_id'])

remained_lncRNA.to_csv("./human/no_trans.csv", index=False)

print("Processing complete. Results saved.")


Processing complete. Results saved.


In [14]:
# Mouse

import pandas as pd
import os
import re

# Set the directory containing ensembl transcript data
ensembl_dir = "../../reference_lncRNA/mouse/transcript/ensembl/"

# Read LncBook and NONCODE transcript files
cols = ['gene_id', 'transcript_id']
noncodev5_trans = pd.read_csv('../../reference_lncRNA/mouse/transcript/NONCODEv5_mouse_mm10_lncRNA_trans.txt', sep='\t', header=None, names=cols)

# Read lncRNA ID data
lncRNA = pd.read_csv('../../data/LPI/mouse/lncRNA.csv')
lncRNA = lncRNA[['lncRNA_id','gene_id','gene_name', 'identifier']]

# Initialize remaining lncRNA list
remained_lncRNA = lncRNA[['lncRNA_id','gene_id','gene_name', 'identifier']].copy()

# Function to remove matched rows
def update_remained_lncRNA(df, matched_df):
    """Remove matched lncRNA_id from the remaining lncRNA list"""
    updated_df = df[~df['lncRNA_id'].isin(matched_df['lncRNA_id'])]
    return updated_df[['lncRNA_id','gene_id','gene_name', 'identifier']]

# Store results
results = []

# Get transcript by NONCODE (v5)
trans_lnc_noncodev5 = pd.merge(remained_lncRNA, noncodev5_trans, left_on='identifier', right_on='gene_id', how='inner')
results.append(trans_lnc_noncodev5)
remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_noncodev5)

# Extract version numbers and sort filenames in descending order
def extract_version(filename):
    #match = re.search(r'GRCm38\.(\d+)_trans\.txt', filename)
    match = re.search(r"v(\d+)", filename) # ensembl
    return int(match.group(1)) if match else -1  # Extract version number, default to -1 if no match

ensembl_files = [f for f in os.listdir(ensembl_dir) if f.endswith(".txt")]
sorted_ensembl_files = sorted(ensembl_files, key=extract_version, reverse=True)  # Sort by version number (descending)

# Iterate through sorted ensembl transcript files
for txt_file in sorted_ensembl_files:
    file_path = os.path.join(ensembl_dir, txt_file)

    # Read ensembl transcript file
    ensembl_trans = pd.read_csv(
        file_path, sep='\t', header=None,
        names=['gene_id', 'gene_name', 'transcript_id']
    )

    trans_lnc_ensembl = pd.merge(
        remained_lncRNA,
        ensembl_trans[['gene_id', 'transcript_id']],
        left_on='identifier', right_on='gene_id', how='inner'
    )
    results.append(trans_lnc_ensembl)
    remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_ensembl)
    
for txt_file in sorted_ensembl_files:
    file_path = os.path.join(ensembl_dir, txt_file)

    # Read ensembl transcript file
    ensembl_trans = pd.read_csv(
        file_path, sep='\t', header=None,
        names=['gene_id', 'gene_name', 'transcript_id']
    )
    
    trans_lnc_symbol = pd.merge(
        remained_lncRNA,
        ensembl_trans[['gene_name', 'transcript_id']],
        left_on='identifier', right_on='gene_name', how='inner'
    )
    results.append(trans_lnc_symbol)

    # --- Update remained set ---
    remained_lncRNA = update_remained_lncRNA(remained_lncRNA, trans_lnc_symbol)

remained_lncRNA.to_csv("no_trans.csv", index=False)

# Combine all results into a single DataFrame
trans_lnc = pd.concat(results, ignore_index=True)
trans_lnc = trans_lnc[['lncRNA_id', 'transcript_id']]

# Save results to file
trans_lnc.drop_duplicates().to_csv('./mouse/lnc_trans.csv', index=False, header=None)

remained_lncRNA.to_csv("./mouse/no_trans.csv", index=False)

print("Processing complete. Results saved.")


Processing complete. Results saved.


### Step2 获取转录本序列，并过滤掉长度超过20000nt的转录本。

In [30]:
# Human
from Bio import SeqIO
import pandas as pd
import os
import re

# Define sequence length limit
MAX_SEQUENCE_LENGTH = 20000  # Remove genes if any transcript exceeds this length

# Load the mapping file and initialize dictionaries
def load_transcript_ids(mapping_file):
    df = pd.read_csv(mapping_file, dtype=str, header=None, names=['lncRNA_id', 'transcript_id'])
    
    gene_to_transcripts = {}  # Store mapping from gene ID to transcript ID list
    transcript_to_gene = {}   # Store mapping from transcript ID to gene ID

    for _, row in df.iterrows():
        gene_id = row['lncRNA_id'].strip()
        transcript_id = row['transcript_id'].strip()

        if gene_id not in gene_to_transcripts:
            gene_to_transcripts[gene_id] = set()
        gene_to_transcripts[gene_id].add(transcript_id)
        transcript_to_gene[transcript_id] = gene_id

    return gene_to_transcripts, transcript_to_gene

# Fetch sequences for transcript IDs from a list of FASTA files
def fetch_sequences(fasta_files, gene_to_transcripts):
    found_transcripts = {}  # Store found transcript sequences
    missing_genes = set()   # Store genes whose sequences are missing
    oversized_genes = set() # Store genes that contain transcripts exceeding MAX_SEQUENCE_LENGTH

    # Create a set of all transcript IDs that need to be found
    all_transcript_ids = {tid for tids in gene_to_transcripts.values() for tid in tids}
    needed_ids = all_transcript_ids.copy()  # Copy set to track missing IDs

    for file in fasta_files:
        print(f"Checking file: {file}")  # Debugging output
        if not needed_ids:  # Stop early if all sequences have been found
            break
        for record in SeqIO.parse(file, "fasta"):
            if record.id in needed_ids:
                if len(record.seq) > MAX_SEQUENCE_LENGTH:
                    oversized_genes.add(transcript_to_gene[record.id])  # Mark gene for removal
                else:
                    found_transcripts[record.id] = record.seq  # Store valid transcript
                needed_ids.remove(record.id)  # Remove found ID from the set

    # Identify genes for which all transcripts are missing
    for gene_id, transcript_ids in gene_to_transcripts.items():
        if not all(tid in found_transcripts for tid in transcript_ids):  # If any transcripts of a gene are missing
            missing_genes.add(gene_id)

    # Combine genes to remove: those missing and those with oversized transcripts
    genes_to_remove = missing_genes.union(oversized_genes)

    # Remove the affected genes and their transcripts
    for gene_id in genes_to_remove:
        for tid in gene_to_transcripts[gene_id]:  # Iterate through all transcripts of the gene
            found_transcripts.pop(tid, None)  # Ensure all associated transcripts are removed

    # Filter out removed genes from the gene-to-transcript dictionary
    filtered_gene_to_transcripts = {gene: trans for gene, trans in gene_to_transcripts.items() if gene not in genes_to_remove}

    print(f"Total found sequences: {len(found_transcripts)}")
    print(f"Total missing genes removed: {len(missing_genes)}")
    print(f"Total oversized genes removed: {len(oversized_genes)}")

    return found_transcripts, filtered_gene_to_transcripts

# Write the found sequences to a new FASTA file
def write_fasta(sequences, output_file):
    with open(output_file, "w") as f:
        for trans_id, seq in sequences.items():
            SeqIO.write(SeqIO.SeqRecord(seq, id=trans_id, description=""), f, "fasta")
        print(f"Written {len(sequences)} sequences to {output_file}")  # Debugging output

# Write the filtered gene-to-transcript mappings to a new file
def write_filtered_mapping(filtered_gene_to_transcripts, output_mapping_file, sequences):
    with open(output_mapping_file, "w") as f:
        for gene_id, transcript_ids in filtered_gene_to_transcripts.items():
            # keep only transcripts that actually have sequences
            kept = [tid for tid in transcript_ids if tid in sequences]
            for tid in kept:
                f.write(f"{gene_id},{tid}\n")

# File paths and execution
mapping_file = "./human/lnc_trans.csv"
base_directory = "../../reference_lncRNA/human/fasta/"
processed_ensembl_dir = "../../reference_lncRNA/human/fasta/processed_ensembl/"

def extract_version(filename):
    match = re.search(r'v(\d+)', filename)
    #match = re.search(r'GRCh38\.(\d+).ncrna_processed\.fa', filename)
    return int(match.group(1)) if match else -1  # Extract version number, default to -1 if no match

# Get all .fa files in processedensembl directory
processed_ensembl_fa_files = [f for f in os.listdir(processed_ensembl_dir) if f.endswith(".fa")]
sorted_ensembl_files = [os.path.join(processed_ensembl_dir, f) for f in sorted(processed_ensembl_fa_files, key=extract_version, reverse=True)]

fasta_files = [
    os.path.join(base_directory, "NONCODEv6_human_processed.fa"),
    os.path.join(base_directory, "NONCODEv5_human_processed.fa")
] + sorted_ensembl_files  # Add dynamically found files

output_fasta_file = "./human/transcript_sequences.fasta"
output_mapping_file = "./human/filtered_lnc_trans.csv"

# Load required transcript IDs and gene mappings
gene_to_transcripts, transcript_to_gene = load_transcript_ids(mapping_file)

# Fetch sequences and remove missing/oversized genes
sequences, filtered_gene_to_transcripts = fetch_sequences(fasta_files, gene_to_transcripts)

# Write the filtered sequences to output FASTA file
write_fasta(sequences, output_fasta_file)

# Write the filtered gene-transcript mappings to a new file
write_filtered_mapping(filtered_gene_to_transcripts, output_mapping_file, sequences)


Checking file: ../../reference_lncRNA/human/fasta/NONCODEv6_human_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/NONCODEv5_human_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v113.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v112.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v111.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v110.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v109.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v108.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v107.ncrna_processed.fa
Checking file: ../../reference_lncRNA/human/fasta/processe

In [31]:
# Mouse
from Bio import SeqIO
import pandas as pd
import os
import re

# Define sequence length limit
MAX_SEQUENCE_LENGTH = 20000  # Remove genes if any transcript exceeds this length

# Load the mapping file and initialize dictionaries
def load_transcript_ids(mapping_file):
    df = pd.read_csv(mapping_file, dtype=str, header=None, names=['lncRNA_id', 'transcript_id'])
    
    gene_to_transcripts = {}  # Store mapping from gene ID to transcript ID list
    transcript_to_gene = {}   # Store mapping from transcript ID to gene ID

    for _, row in df.iterrows():
        gene_id = row['lncRNA_id'].strip()
        transcript_id = row['transcript_id'].strip()

        if gene_id not in gene_to_transcripts:
            gene_to_transcripts[gene_id] = set()
        gene_to_transcripts[gene_id].add(transcript_id)
        transcript_to_gene[transcript_id] = gene_id

    return gene_to_transcripts, transcript_to_gene

# Fetch sequences for transcript IDs from a list of FASTA files
def fetch_sequences(fasta_files, gene_to_transcripts):
    found_transcripts = {}  # Store found transcript sequences
    missing_genes = set()   # Store genes whose sequences are missing
    oversized_genes = set() # Store genes that contain transcripts exceeding MAX_SEQUENCE_LENGTH

    # Create a set of all transcript IDs that need to be found
    all_transcript_ids = {tid for tids in gene_to_transcripts.values() for tid in tids}
    needed_ids = all_transcript_ids.copy()  # Copy set to track missing IDs

    for file in fasta_files:
        print(f"Checking file: {file}")  # Debugging output
        if not needed_ids:  # Stop early if all sequences have been found
            break
        for record in SeqIO.parse(file, "fasta"):
            if record.id in needed_ids:
                if len(record.seq) > MAX_SEQUENCE_LENGTH:
                    oversized_genes.add(transcript_to_gene[record.id])  # Mark gene for removal
                else:
                    found_transcripts[record.id] = record.seq  # Store valid transcript
                needed_ids.remove(record.id)  # Remove found ID from the set

    # Identify genes for which all transcripts are missing
    for gene_id, transcript_ids in gene_to_transcripts.items():
        if not all(tid in found_transcripts for tid in transcript_ids):  # If any transcripts of a gene are missing
            missing_genes.add(gene_id)

    # Combine genes to remove: those missing and those with oversized transcripts
    genes_to_remove = missing_genes.union(oversized_genes)

    # Remove the affected genes and their transcripts
    for gene_id in genes_to_remove:
        for tid in gene_to_transcripts[gene_id]:  # Iterate through all transcripts of the gene
            found_transcripts.pop(tid, None)  # Ensure all associated transcripts are removed

    # Filter out removed genes from the gene-to-transcript dictionary
    filtered_gene_to_transcripts = {gene: trans for gene, trans in gene_to_transcripts.items() if gene not in genes_to_remove}

    print(f"Total found sequences: {len(found_transcripts)}")
    print(f"Total missing genes removed: {len(missing_genes)}")
    print(f"Total oversized genes removed: {len(oversized_genes)}")

    return found_transcripts, filtered_gene_to_transcripts

# Write the found sequences to a new FASTA file
def write_fasta(sequences, output_file):
    with open(output_file, "w") as f:
        for trans_id, seq in sequences.items():
            SeqIO.write(SeqIO.SeqRecord(seq, id=trans_id, description=""), f, "fasta")
        print(f"Written {len(sequences)} sequences to {output_file}")  # Debugging output

# Write the filtered gene-to-transcript mappings to a new file
def write_filtered_mapping(filtered_gene_to_transcripts, output_mapping_file, sequences):
    with open(output_mapping_file, "w") as f:
        for gene_id, transcript_ids in filtered_gene_to_transcripts.items():
            # keep only transcripts that actually have sequences
            kept = [tid for tid in transcript_ids if tid in sequences]
            for tid in kept:
                f.write(f"{gene_id},{tid}\n")

# File paths and execution
mapping_file = "./mouse/lnc_trans.csv"
base_directory = "../../reference_lncRNA/mouse/fasta/"
processed_ensembl_dir = "../../reference_lncRNA/mouse/fasta/processed_ensembl/"

def extract_version(filename):
    match = re.search(r'v(\d+)', filename)
    #match = re.search(r'GRCh38\.(\d+).ncrna_processed\.fa', filename)
    return int(match.group(1)) if match else -1  # Extract version number, default to -1 if no match

# Get all .fa files in processedensembl directory
processed_ensembl_fa_files = [f for f in os.listdir(processed_ensembl_dir) if f.endswith(".fa")]
sorted_ensembl_files = [os.path.join(processed_ensembl_dir, f) for f in sorted(processed_ensembl_fa_files, key=extract_version, reverse=True)]

fasta_files = [
    os.path.join(base_directory, "NONCODEv6_mouse_processed.fa"),
    os.path.join(base_directory, "NONCODEv5_mouse_processed.fa")
] + sorted_ensembl_files  # Add dynamically found files

output_fasta_file = "./mouse/transcript_sequences.fasta"
output_mapping_file = "./mouse/filtered_lnc_trans.csv"

# Load required transcript IDs and gene mappings
gene_to_transcripts, transcript_to_gene = load_transcript_ids(mapping_file)

# Fetch sequences and remove missing/oversized genes
sequences, filtered_gene_to_transcripts = fetch_sequences(fasta_files, gene_to_transcripts)

# Write the filtered sequences to output FASTA file
write_fasta(sequences, output_fasta_file)

# Write the filtered gene-transcript mappings to a new file
write_filtered_mapping(filtered_gene_to_transcripts, output_mapping_file, sequences)



Checking file: ../../reference_lncRNA/mouse/fasta/NONCODEv6_mouse_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/NONCODEv5_mouse_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v100.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v99.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v98.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v97.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v96.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v95.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v94.ncrna_processed.fa
Checking file: ../../reference_lncRNA/mouse/fasta/processed_ense

### Step3 Calculate MFE

In [ ]:
! ./cal_MFE.sh ./human/transcript_sequences.fasta ./human/trans_MFE.csv

In [ ]:
! ./cal_MFE.sh ./mouse/transcript_sequences.fasta ./mouse/trans_MFE.csv

### Step4 Calculate GIC score

In [ ]:
# Human

import math
import pandas as pd
from tqdm import tqdm


# record all triplet combinations
BASES = ['a', 't', 'c', 'g']
TRIPLETS = []
for B1 in BASES:
    for B2 in BASES:
        for B3 in BASES:
            TRIPLETS.append(B1+B2+B3)

mers = {} #construct a list of triplets that are initialized to 0

# obtain the corresponding relationship between lncRNA and transcript
def readlnc_transID(filename):
    lnc_trans = pd.read_csv(filename, sep='\t', header=None, names=['lncRNA_id', 'transcript_id'])
    #Create the mapping relationship between genes and transcripts
    lnc_trans_mapping = lnc_trans.groupby('lncRNA_id')['transcript_id'].aggregate(list).to_dict()

    return lnc_trans_mapping

# Get the secondary structure minimum free energy of each transcript of lncRNA
def readlnc_mfe(filename):
    trans_mfe = pd.read_csv(filename)
    #Create the mapping relationship between transcripts and MFE
    trans_mfe_mapping = pd.Series(trans_mfe['MFE'].values, index=trans_mfe['transcript_id']).to_dict()

    return trans_mfe_mapping

# Get transcript sequence
def readfasta(filename):
    f = open(filename,'r')
    res = {} #Preserve transcripts and their sequence
    for line in f:
        if line.startswith('>'):
            line = line.strip()
            ID = line.split('>',2)[1]
            res[ID] = ''
        else:
            res[ID] += line
    return res

#sliding window
def slidingWindow(seq, l, win, step=1): 
    length = l
    mod = divmod((length-win), step)[1]
    if (win >= length):
        return seq
    else:
        start = 0
        end = win
        fragments = []
        while (len(seq[start:end]) == win):
            fragments.append(seq[start:end])
            start += step
            end += step
        if (mod > 0):
            fragments.append(seq[(length-win):])
        return fragments
 
#calculate the frequency of the triplet
def stat3mer(seq, l):   
    freq = {}
    for item in TRIPLETS:
        mers[item] = 0
    num3mer = float(l-2)
    all3mer = slidingWindow(seq, l, win=3, step=1)
    for i in set(TRIPLETS):
        mers[i] = all3mer.count(i)
    for triplet in TRIPLETS:
        freq[triplet] = mers[triplet]/num3mer
    return freq


if __name__ == '__main__':

    lnc_trans_path = './human/filtered_lnc_trans.txt'  
    MFE_path = './human/trans_MFE.csv'
    seq_path = './human/transcript_sequences.fasta'  

    lnc_transID = readlnc_transID(lnc_trans_path)
    trans_mfe = readlnc_mfe(MFE_path)
    trans_seq = readfasta(seq_path)

    # parameter Settings of GIC
    LRMODEL_FEATURES_7 = ['intercept', 'length', 'mfe/L', 'cga', 'gcg', 'tcg', 'acg', 'tca']
    LRMODEL_COEFS_7 = [0.7417, 2.612e-04, 4.295, 48.66, 15.64, 76.23, -1.113, -60.29]
    LRMODEL_7 = dict(zip(LRMODEL_FEATURES_7, LRMODEL_COEFS_7))

    #calculated the eigenvalues of each transcript
    features_trans = {}  #save the features of each transcript
    for id, seq in tqdm(trans_seq.items(), desc="Processing Transcripts"):
        if id not in trans_mfe:
            continue
        feature = {}
        seq = seq.replace('\n','')
        seq = seq.lower()
        length = len(seq)
        feature['trans'] = id
        feature['length'] = length
        feature['mfe/L'] = trans_mfe[id]/length
        freq = stat3mer(seq, length)
        for item in TRIPLETS:
            feature[item] = freq[item]
        # tmp = LRMODEL_7['intercept']+sum([feature[_]*LRMODEL_7[_] for _ in LRMODEL_FEATURES_7[1:]])
        # gic = math.exp(tmp)/(math.exp(tmp)+1)
        # feature['GIC_7'] = gic
        features_trans[id] = feature
    
    #calculated the eigenvalues of each lncRNA
    lncRNA_GIC_score = {}  ##save the GIC score of each lncRNA
    for lnc,transcripts in lnc_transID.items():
        feature = {} #save the features of lncRNA gene
        feature['length'] = 0 # initialize the value of each feature to 0
        feature['mfe/L'] = 0
        for item in TRIPLETS:
            feature[item] = 0
    
        for trans in transcripts:
            if trans not in features_trans:
                feature = {}
                break
            else:
                tranfeature = features_trans[trans]
                feature['length'] += tranfeature['length']
                feature['mfe/L']  += tranfeature['mfe/L']
                for item in TRIPLETS:
                    feature[item] += tranfeature[item]
        if len(feature)>0:
            feature['lncRNA_id'] = lnc
            feature['length'] = feature['length']/len(transcripts)
            feature['mfe/L'] = feature['mfe/L']/len(transcripts)
            for item in TRIPLETS:
                feature[item] =  feature[item]/len(transcripts)
            tmp = LRMODEL_7['intercept']+sum([feature[_]*LRMODEL_7[_] for _ in LRMODEL_FEATURES_7[1:]])
            gic = math.exp(tmp)/(math.exp(tmp)+1)
            feature['GIC_score'] = gic
            lncRNA_GIC_score[lnc] = feature['GIC_score']

    # sort all LncRNAs according to GIC scores
    lncRNA_GIC_score = sorted(lncRNA_GIC_score.items(),key=lambda d:d[1],reverse=False)
    
    # Convert the lncRNA GIC scores to a DataFrame
    lncRNA_scores_df_sorted = pd.DataFrame(lncRNA_GIC_score, columns=['lncRNA_id', 'GIC_score'])

    # Write the sorted results to a CSV file
    sorted_GIC_path = './human/sorted_GIC_score.csv'
    lncRNA_scores_df_sorted.to_csv(sorted_GIC_path, index=False)

    print("GIC scores have been calculated and saved successfully.")
    print(f"Output file is located at: {sorted_GIC_path}")

In [ ]:
# Mouse

import math
import pandas as pd
from tqdm import tqdm


# record all triplet combinations
BASES = ['a', 't', 'c', 'g']
TRIPLETS = []
for B1 in BASES:
    for B2 in BASES:
        for B3 in BASES:
            TRIPLETS.append(B1+B2+B3)

mers = {} #construct a list of triplets that are initialized to 0

# obtain the corresponding relationship between lncRNA and transcript
def readlnc_transID(filename):
    lnc_trans = pd.read_csv(filename, sep='\t', header=None, names=['lncRNA_id', 'transcript_id'])
    #Create the mapping relationship between genes and transcripts
    lnc_trans_mapping = lnc_trans.groupby('lncRNA_id')['transcript_id'].aggregate(list).to_dict()

    return lnc_trans_mapping

# Get the secondary structure minimum free energy of each transcript of lncRNA
def readlnc_mfe(filename):
    trans_mfe = pd.read_csv(filename)
    #Create the mapping relationship between transcripts and MFE
    trans_mfe_mapping = pd.Series(trans_mfe['MFE'].values, index=trans_mfe['transcript_id']).to_dict()

    return trans_mfe_mapping

# Get transcript sequence
def readfasta(filename):
    f = open(filename,'r')
    res = {} #Preserve transcripts and their sequence
    for line in f:
        if line.startswith('>'):
            line = line.strip()
            ID = line.split('>',2)[1]
            res[ID] = ''
        else:
            res[ID] += line
    return res

#sliding window
def slidingWindow(seq, l, win, step=1): 
    length = l
    mod = divmod((length-win), step)[1]
    if (win >= length):
        return seq
    else:
        start = 0
        end = win
        fragments = []
        while (len(seq[start:end]) == win):
            fragments.append(seq[start:end])
            start += step
            end += step
        if (mod > 0):
            fragments.append(seq[(length-win):])
        return fragments
 
#calculate the frequency of the triplet
def stat3mer(seq, l):   
    freq = {}
    for item in TRIPLETS:
        mers[item] = 0
    num3mer = float(l-2)
    all3mer = slidingWindow(seq, l, win=3, step=1)
    for i in set(TRIPLETS):
        mers[i] = all3mer.count(i)
    for triplet in TRIPLETS:
        freq[triplet] = mers[triplet]/num3mer
    return freq


if __name__ == '__main__':

    lnc_trans_path = './mouse/filtered_lnc_trans.txt'  
    MFE_path = './mouse/trans_MFE.csv'
    seq_path = './mouse/transcript_sequences.fasta'  

    lnc_transID = readlnc_transID(lnc_trans_path)
    trans_mfe = readlnc_mfe(MFE_path)
    trans_seq = readfasta(seq_path)

    # parameter Settings of GIC
    LRMODEL_FEATURES_7 = ['intercept', 'length', 'mfe/L', 'cga', 'gcg', 'tcg', 'acg', 'tca']
    LRMODEL_COEFS_7 = [0.7417, 2.612e-04, 4.295, 48.66, 15.64, 76.23, -1.113, -60.29]
    LRMODEL_7 = dict(zip(LRMODEL_FEATURES_7, LRMODEL_COEFS_7))

    #calculated the eigenvalues of each transcript
    features_trans = {}  #save the features of each transcript
    for id, seq in tqdm(trans_seq.items(), desc="Processing Transcripts"):
        if id not in trans_mfe:
            continue
        feature = {}
        seq = seq.replace('\n','')
        seq = seq.lower()
        length = len(seq)
        feature['trans'] = id
        feature['length'] = length
        feature['mfe/L'] = trans_mfe[id]/length
        freq = stat3mer(seq, length)
        for item in TRIPLETS:
            feature[item] = freq[item]
        # tmp = LRMODEL_7['intercept']+sum([feature[_]*LRMODEL_7[_] for _ in LRMODEL_FEATURES_7[1:]])
        # gic = math.exp(tmp)/(math.exp(tmp)+1)
        # feature['GIC_7'] = gic
        features_trans[id] = feature
    
    #calculated the eigenvalues of each lncRNA
    lncRNA_GIC_score = {}  ##save the GIC score of each lncRNA
    for lnc,transcripts in lnc_transID.items():
        feature = {} #save the features of lncRNA gene
        feature['length'] = 0 # initialize the value of each feature to 0
        feature['mfe/L'] = 0
        for item in TRIPLETS:
            feature[item] = 0
    
        for trans in transcripts:
            if trans not in features_trans:
                feature = {}
                break
            else:
                tranfeature = features_trans[trans]
                feature['length'] += tranfeature['length']
                feature['mfe/L']  += tranfeature['mfe/L']
                for item in TRIPLETS:
                    feature[item] += tranfeature[item]
        if len(feature)>0:
            feature['lncRNA_id'] = lnc
            feature['length'] = feature['length']/len(transcripts)
            feature['mfe/L'] = feature['mfe/L']/len(transcripts)
            for item in TRIPLETS:
                feature[item] =  feature[item]/len(transcripts)
            tmp = LRMODEL_7['intercept']+sum([feature[_]*LRMODEL_7[_] for _ in LRMODEL_FEATURES_7[1:]])
            gic = math.exp(tmp)/(math.exp(tmp)+1)
            feature['GIC_score'] = gic
            lncRNA_GIC_score[lnc] = feature['GIC_score']

    # sort all LncRNAs according to GIC scores
    lncRNA_GIC_score = sorted(lncRNA_GIC_score.items(),key=lambda d:d[1],reverse=False)
    
    # Convert the lncRNA GIC scores to a DataFrame
    lncRNA_scores_df_sorted = pd.DataFrame(lncRNA_GIC_score, columns=['lncRNA_id', 'GIC_score'])

    # Write the sorted results to a CSV file
    sorted_GIC_path = './mouse/sorted_GIC_score.csv'
    lncRNA_scores_df_sorted.to_csv(sorted_GIC_path, index=False)

    print("GIC scores have been calculated and saved successfully.")
    print(f"Output file is located at: {sorted_GIC_path}")

### Step5 Get essential lncRNA samples and non-essential lncRNA samples

In [ ]:
# Human
import pandas as pd

# Read the lncRNA data
ess_lnc = pd.read_csv("../../data/benchmark/human/human_esslnc.csv")
lncRNA = pd.read_csv("../../data/LPI/human/lncRNA.csv")

ess_lnc['Noncode_id'] = ess_lnc['Noncode_id'].str.split('.').str[0]

def is_essential(id, name):
    if id != '-' and any(ess_lnc[col].isin([id]).any() for col in ['Noncode_id', 'ensembl_id']):
        return 1
    if name != '-' and any(ess_lnc[col].isin([name]).any() for col in ['gene_name', 'lib_id']):
        return 1
    return 0

lncRNA['essential'] = lncRNA.apply(lambda row: is_essential(row['gene_id'], row['gene_name']), axis=1)

ess_lnc = lncRNA[lncRNA['essential'] == 1]
ess_lnc = ess_lnc[['lncRNA_ID']]

ess_lnc.to_csv('../../data/benchmark/human/ess_lnc.csv', index=False)

lnc_GIC = pd.read_csv("sorted_GIC_score.csv")

annotated_lnc = pd.read_csv("../../Annotate/human/valid_heart_annotation.csv")

unlabel_lnc = lnc_GIC[~lnc_GIC['lncRNA_ID'].isin(ess_lnc['lncRNA_ID'])]
unlabel_lnc = unlabel_lnc[unlabel_lnc['lncRNA_ID'].isin(annotated_lnc['lncRNA_ID'])]

ess_counts = ess_lnc.shape[0]

noness_lnc = unlabel_lnc[:ess_counts]

noness_lnc = noness_lnc[['lncRNA_ID']]

noness_lnc.to_csv("../../data/benchmark/human/noness_lnc.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/benchmark/human/human_esslnc.csv'

In [ ]:
# Mouse
import pandas as pd

# Read the lncRNA data
ess_lnc = pd.read_csv("../../data/benchmark/mouse/mouse_esslnc.csv")
lncRNA = pd.read_csv("../../data/LPI/mouse/lncRNA.csv")

ess_lnc['Noncode_id'] = ess_lnc['Noncode_id'].str.split('.').str[0]

def is_essential(id, name):
    if id != '-' and any(ess_lnc[col].isin([id]).any() for col in ['Noncode_id', 'ensembl_id']):
        return 1
    if name != '-' and any(ess_lnc[col].isin([name]).any() for col in ['gene_name', 'lib_id']):
        return 1
    return 0

lncRNA['essential'] = lncRNA.apply(lambda row: is_essential(row['gene_id'], row['gene_name']), axis=1)

ess_lnc = lncRNA[lncRNA['essential'] == 1]
ess_lnc = ess_lnc[['lncRNA_ID']]

ess_lnc.to_csv('../../data/benchmark/mouse/ess_lnc.csv', index=False)

lnc_GIC = pd.read_csv("sorted_GIC_score.csv")

annotated_lnc = pd.read_csv("../../Annotate/mouse/valid_heart_annotation.csv")

unlabel_lnc = lnc_GIC[~lnc_GIC['lncRNA_ID'].isin(ess_lnc['lncRNA_ID'])]
unlabel_lnc = unlabel_lnc[unlabel_lnc['lncRNA_ID'].isin(annotated_lnc['lncRNA_ID'])]

ess_counts = ess_lnc.shape[0]

noness_lnc = unlabel_lnc[:ess_counts]

noness_lnc = noness_lnc[['lncRNA_ID']]

noness_lnc.to_csv("../../data/benchmark/mouse/noness_lnc.csv", index=False)

从这里开始，是为没有计算MFE的转录本重新计算的代码，使用完后先存档，然后删除

先提取需要计算的转录本

In [33]:
import pandas as pd

# ----------------------------
# Input / output file paths
# ----------------------------
csv1 = "./mouse/filtered_lnc_trans.csv"   # your first csv
csv2 = "./mouse/trans_MFE.csv"   # your second csv
out_file = "./mouse/need_cal.csv"

# ----------------------------
# Load CSVs
# ----------------------------
df1 = pd.read_csv(csv1, dtype=str)   # keep as string
df2 = pd.read_csv(csv2, dtype=str)

# ----------------------------
# Extract key columns
# ----------------------------
col1_csv1 = df1.columns[1]   # second column in csv1
col0_csv2 = df2.columns[0]   # first column in csv2

# ----------------------------
# Build a set of reference values from csv2
# ----------------------------
ref_values = set(df2[col0_csv2].dropna().astype(str).str.strip())

# ----------------------------
# Filter rows from csv1 where second column is NOT in ref_values
# ----------------------------
mask = ~df1[col1_csv1].astype(str).str.strip().isin(ref_values)
filtered_df = df1.loc[mask].copy()

# ----------------------------
# Save the result
# ----------------------------
filtered_df.to_csv(out_file, index=False)

print(f"Total rows in file1: {len(df1)}")
print(f"Rows kept (not in file2): {len(filtered_df)}")
print(f"Filtered file saved to: {out_file}")


Total rows in file1: 57312
Rows kept (not in file2): 20
Filtered file saved to: ./mouse/need_cal.csv


提取这些转录本的序列信息

In [34]:
# Human
from Bio import SeqIO
import pandas as pd
import os
import re

# ----------------------------
# Load transcript IDs from mapping file
# mapping_file columns (no header): lncRNA_id, transcript_id
# We only need the set of transcript_ids to fetch
# ----------------------------
def load_transcript_ids(mapping_file):
    df = pd.read_csv(mapping_file, dtype=str, header=None, names=['lncRNA_id', 'transcript_id'])
    df = df.dropna(subset=['transcript_id'])
    df['transcript_id'] = df['transcript_id'].str.strip()
    transcript_ids = set(df['transcript_id'])
    return transcript_ids

# ----------------------------
# Fetch sequences for the requested transcript IDs
# Search multiple FASTA files in order; stop early when all found
# ----------------------------
def fetch_sequences(fasta_files, transcript_ids):
    found = {}           # transcript_id -> Seq object
    needed = set(transcript_ids)  # IDs still needed

    for fa in fasta_files:
        print(f"[INFO] Scanning: {fa}")
        if not needed:
            break
        for record in SeqIO.parse(fa, "fasta"):
            tid = record.id
            if tid in needed:
                found[tid] = record.seq
                needed.remove(tid)

    print(f"[SUMMARY] Found {len(found)} sequences; {len(needed)} transcripts not found.")
    return found

# ----------------------------
# Write sequences to a FASTA file
# ----------------------------
def write_fasta(sequences, output_file):
    count = 0
    with open(output_file, "w") as f:
        for tid, seq in sequences.items():
            SeqIO.write(SeqIO.SeqRecord(seq, id=tid, description=""), f, "fasta")
            count += 1
    print(f"[DONE] Written {count} sequences to {output_file}")

# ----------------------------
# File paths and execution
# ----------------------------
mapping_file = "./human/need_cal.csv"
base_directory = "../../reference_lncRNA/human/fasta/"
processed_ensembl_dir = "../../reference_lncRNA/human/fasta/processed_ensembl/"

def extract_version(filename):
    # Extract version number like "...v12..." -> 12; return -1 if not matched
    m = re.search(r'v(\d+)', filename)
    return int(m.group(1)) if m else -1

# Collect processed Ensembl FASTA files (sorted by version desc)
processed_ensembl_fa_files = [f for f in os.listdir(processed_ensembl_dir) if f.endswith(".fa")]
sorted_ensembl_files = [
    os.path.join(processed_ensembl_dir, f)
    for f in sorted(processed_ensembl_fa_files, key=extract_version, reverse=True)
]

# Search order: NONCODE v6 -> NONCODE v5 -> Ensembl processed (newest first)
fasta_files = [
    os.path.join(base_directory, "NONCODEv6_human_processed.fa"),
    os.path.join(base_directory, "NONCODEv5_human_processed.fa"),
] + sorted_ensembl_files

output_fasta_file = "./human/need_cal.fasta"

# ----------------------------
# Run
# ----------------------------
transcript_ids = load_transcript_ids(mapping_file)
sequences = fetch_sequences(fasta_files, transcript_ids)
write_fasta(sequences, output_fasta_file)



[INFO] Scanning: ../../reference_lncRNA/human/fasta/NONCODEv6_human_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/NONCODEv5_human_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v113.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v112.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v111.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v110.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v109.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v108.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/human/fasta/processed_ensembl/Homo_sapiens.GRCh38.v107.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/

In [ ]:
# Mouse
from Bio import SeqIO
import pandas as pd
import os
import re

# ----------------------------
# Load transcript IDs from mapping file
# mapping_file columns (no header): lncRNA_id, transcript_id
# We only need the set of transcript_ids to fetch
# ----------------------------
def load_transcript_ids(mapping_file):
    df = pd.read_csv(mapping_file, dtype=str, header=None, names=['lncRNA_id', 'transcript_id'])
    df = df.dropna(subset=['transcript_id'])
    df['transcript_id'] = df['transcript_id'].str.strip()
    transcript_ids = set(df['transcript_id'])
    return transcript_ids

# ----------------------------
# Fetch sequences for the requested transcript IDs
# Search multiple FASTA files in order; stop early when all found
# ----------------------------
def fetch_sequences(fasta_files, transcript_ids):
    found = {}                 # transcript_id -> Seq object
    needed = set(transcript_ids)

    for fa in fasta_files:
        print(f"[INFO] Scanning: {fa}")
        if not needed:
            break
        for record in SeqIO.parse(fa, "fasta"):
            tid = record.id
            if tid in needed:
                found[tid] = record.seq
                needed.remove(tid)

    print(f"[SUMMARY] Found {len(found)} sequences; {len(needed)} transcripts not found.")
    return found

# ----------------------------
# Write sequences to a FASTA file
# ----------------------------
def write_fasta(sequences, output_file):
    count = 0
    with open(output_file, "w") as f:
        for tid, seq in sequences.items():
            SeqIO.write(SeqIO.SeqRecord(seq, id=tid, description=""), f, "fasta")
            count += 1
    print(f"[DONE] Written {count} sequences to {output_file}")

# ----------------------------
# File paths and execution
# ----------------------------
mapping_file = "./mouse/need_cal.csv"
base_directory = "../../reference_lncRNA/mouse/fasta/"
processed_ensembl_dir = "../../reference_lncRNA/mouse/fasta/processed_ensembl/"

def extract_version(filename):
    # Extract version number like "...v12..." -> 12; return -1 if not matched
    m = re.search(r'v(\d+)', filename)
    return int(m.group(1)) if m else -1

# Collect processed Ensembl FASTA files (sorted by version desc)
processed_ensembl_fa_files = [f for f in os.listdir(processed_ensembl_dir) if f.endswith(".fa")]
sorted_ensembl_files = [
    os.path.join(processed_ensembl_dir, f)
    for f in sorted(processed_ensembl_fa_files, key=extract_version, reverse=True)
]

# Search order: NONCODE v6 -> NONCODE v5 -> Ensembl processed (newest first)
fasta_files = [
    os.path.join(base_directory, "NONCODEv6_mouse_processed.fa"),
    os.path.join(base_directory, "NONCODEv5_mouse_processed.fa"),
] + sorted_ensembl_files

output_fasta_file = "./mouse/need_cal.fasta"

# ----------------------------
# Run
# ----------------------------
transcript_ids = load_transcript_ids(mapping_file)
sequences = fetch_sequences(fasta_files, transcript_ids)
write_fasta(sequences, output_fasta_file)



[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/NONCODEv6_mouse_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/NONCODEv5_mouse_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v100.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v99.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v98.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v97.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v96.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v95.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/fasta/processed_ensembl/Mus_musculus.GRCm38.v94.ncrna_processed.fa
[INFO] Scanning: ../../reference_lncRNA/mouse/

: 

用新生成的fasta文件计算MFE,将新的MFE补充回去